# CI/CD for a KFP pipeline

**Learning Objectives:**
1. Learn how to create a custom Cloud Build builder to pilote CAIP Pipelines
1. Learn how to write a Cloud Build config file to build and push all the artifacts for a KFP
1. Learn how to setup a Cloud Build Github trigger to rebuild the KFP

In this lab you will walk through authoring of a **Cloud Build** CI/CD workflow that automatically builds and deploys a KFP pipeline. You will also integrate your workflow with **GitHub** by setting up a trigger that starts the  workflow when a new tag is applied to the **GitHub** repo hosting the pipeline's code.




## Configuring environment settings

Update  the `ENDPOINT` constant with the settings reflecting your lab environment. 

The endpoint to the AI Platform Pipelines instance can be found on the [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console.

1. Open the *SETTINGS* for your instance
2. Use the value of the `host` variable in the *Connect to this Kubeflow Pipelines instance from a Python client via Kubeflow Pipelines SKD* section of the *SETTINGS* window.

In [1]:
ENDPOINT = 'https://164002aec59f7c0d-dot-us-central1.pipelines.googleusercontent.com/'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

## Creating the KFP CLI builder



### Exercise

In the cell below, write a docker file that
* Uses `gcr.io/deeplearning-platform-release/base-cpu` as base image
* Install the python package `kfp` with version `0.2.5`
* Starts `/bin/bash` as entrypoint

In [5]:
%%writefile kfp-cli/Dockerfile
FROM gcr.io/deeplearning-platform-release/base-cpu

RUN pip install kfp==0.2.5

ENTRYPOINT '/bin/bash'
# TODO

Overwriting kfp-cli/Dockerfile


### Build the image and push it to your project's **Container Registry**.

In [6]:
IMAGE_NAME='kfp-cli'
TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

### Exercise

In the cell below, use `gcloud builds` to build the `kfp-cli` Docker image and push it to the project gcr.io registry.

In [12]:
!gcloud builds submit --timeout 15m --tag {IMAGE_URI} kfp-cli

Creating temporary tarball archive of 1 file(s) totalling 110 bytes before compression.
Uploading tarball of [kfp-cli] to [gs://qwiklabs-gcp-04-0ad772141888_cloudbuild/source/1636511274.834656-37dacac1451e400fbaa195b687824792.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-04-0ad772141888/locations/global/builds/ae51e111-d503-449e-8578-8a1cddb7dc4f].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/ae51e111-d503-449e-8578-8a1cddb7dc4f?project=1014019713687].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "ae51e111-d503-449e-8578-8a1cddb7dc4f"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-gcp-04-0ad772141888_cloudbuild/source/1636511274.834656-37dacac1451e400fbaa195b687824792.tgz#1636511275245099
Copying gs://qwiklabs-gcp-04-0ad772141888_cloudbuild/source/1636511274.834656-37dacac1451e400fbaa195b687824792.tgz#1636511275245099...
/ [1 files][  230.0 B/  230.0 B]                      

## Understanding the **Cloud Build** workflow.

### Exercise

In the cell below, you'll complete the `cloudbuild.yaml` file describing the CI/CD workflow and prescribing how environment specific settings are abstracted using **Cloud Build** variables.

The CI/CD workflow automates the steps you walked through manually during `lab-02-kfp-pipeline`:
1. Builds the trainer image
1. Builds the base image for custom components
1. Compiles the pipeline
1. Uploads the pipeline to the KFP environment
1. Pushes the trainer and base images to your project's **Container Registry**

Although the KFP backend supports pipeline versioning, this feature has not been yet enable through the **KFP** CLI. As a temporary workaround, in the **Cloud Build** configuration a value of the `TAG_NAME` variable is appended to the name of the pipeline. 

The **Cloud Build** workflow configuration uses both standard and custom [Cloud Build builders](https://cloud.google.com/cloud-build/docs/cloud-builders). The custom builder encapsulates **KFP CLI**. 

In [14]:
IMAGE_NAME='base_image'
TAG='latest'
BASE_IMAGE='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [18]:
%%writefile cloudbuild.yaml

- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/$PROJECT_ID/$_TRAINER_IMAGE_NAME:$TAG_NAME', '.']
  dir: $_PIPELINE_FOLDER/trainer_image

# Build the base image for lightweight components
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/$PROJECT_ID/$_BASE_IMAGE_NAME:$TAG_NAME', '.']
  dir: $_PIPELINE_FOLDER/base_image

# Compile the pipeline
- name: 'gcr.io/$PROJECT_ID/kfp-cli'
  args:
  - '-c'
  - |
    dsl-compile --py $_PIPELINE_DSL --output $_PIPELINE_PACKAGE
  env:
  - 'BASE_IMAGE=gcr.io/$PROJECT_ID/$_BASE_IMAGE_NAME:$TAG_NAME'
  - 'TRAINER_IMAGE=gcr.io/$PROJECT_ID/$_TRAINER_IMAGE_NAME:$TAG_NAME'
  - 'RUNTIME_VERSION=$_RUNTIME_VERSION'
  - 'PYTHON_VERSION=$_PYTHON_VERSION'
  - 'COMPONENT_URL_SEARCH_PREFIX=$_COMPONENT_URL_SEARCH_PREFIX'
  - 'USE_KFP_SA=$_USE_KFP_SA'
  dir: $_PIPELINE_FOLDER/pipeline

 # Upload the pipeline
- name: 'gcr.io/$PROJECT_ID/kfp-cli'
  args:
  - '-c'
  - |
    kfp --endpoint $_ENDPOINT pipeline upload -p ${_PIPELINE_NAME}_$TAG_NAME $_PIPELINE_PACKAGE
  dir: $_PIPELINE_FOLDER/pipeline


# Push the images to Container Registry
images: ['gcr.io/$PROJECT_ID/$_TRAINER_IMAGE_NAME:$TAG_NAME', 'gcr.io/$PROJECT_ID/$_BASE_IMAGE_NAME:$TAG_NAME']

Overwriting cloudbuild.yaml


## Manually triggering CI/CD runs

You can manually trigger **Cloud Build** runs using the `gcloud builds submit` command.

In [22]:
SUBSTITUTIONS="""
_ENDPOINT={},\
_TRAINER_IMAGE_NAME=trainer_image,\
_BASE_IMAGE_NAME=base_image,\
TAG_NAME=test,\
_PIPELINE_FOLDER=.,\
_PIPELINE_DSL=covertype_training_pipeline.py,\
_PIPELINE_PACKAGE=covertype_training_pipeline.yaml,\
_PIPELINE_NAME=covertype_continuous_training,\
_RUNTIME_VERSION=1.15,\
_PYTHON_VERSION=3.7,\
_USE_KFP_SA=True,\
_COMPONENT_URL_SEARCH_PREFIX=https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/
""".format(ENDPOINT).strip()

In [24]:
!gcloud builds submit . --config cloudbuild.yaml --substitutions {SUBSTITUTIONS}

Creating temporary tarball archive of 19 file(s) totalling 88.7 KiB before compression.
Uploading tarball of [.] to [gs://qwiklabs-gcp-04-0ad772141888_cloudbuild/source/1636512971.841364-8c8b453fea9c4938b5ae5b462fff42f7.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-04-0ad772141888/locations/global/builds/8c819e24-2962-4eb7-80a5-04e0776f3d3f].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/8c819e24-2962-4eb7-80a5-04e0776f3d3f?project=1014019713687].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "8c819e24-2962-4eb7-80a5-04e0776f3d3f"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-gcp-04-0ad772141888_cloudbuild/source/1636512971.841364-8c8b453fea9c4938b5ae5b462fff42f7.tgz#1636512972110166
Copying gs://qwiklabs-gcp-04-0ad772141888_cloudbuild/source/1636512971.841364-8c8b453fea9c4938b5ae5b462fff42f7.tgz#1636512972110166...
/ [1 files][ 18.7 KiB/ 18.7 KiB]                            

## Setting up GitHub integration

### Exercise

In this exercise you integrate your CI/CD workflow with **GitHub**, using [Cloud Build GitHub App](https://github.com/marketplace/google-cloud-build). 
You will set up a trigger that starts the CI/CD workflow when a new tag is applied to the **GitHub** repo managing the  pipeline source code. You will use a fork of this repo as your source GitHub repository.

#### Step 1: Create a fork of this repo and reflect changes
1. [Follow the GitHub documentation](https://help.github.com/en/github/getting-started-with-github/fork-a-repo) to fork [this repo](https://github.com/GoogleCloudPlatform/asl-ml-immersion)

2. Open Terminal and Run these command to reflect the change to your repository

```bash
# Setup Git configs with your account infomation
git config --global user.name <git username>
git config --global user.email <git_email>

# Clone your repoository (don't forget to change the username)
git clone https://github.com/<git username>/asl-ml-immersion ~/asl-ml-immersion-labs

# Reflect the changes to your repository
cd ~/asl-ml-immersion-labs
cp ~/asl-ml-immersion/notebooks/kubeflow_pipelines/cicd/labs/cloudbuild.yaml ./notebooks/kubeflow_pipelines/cicd/labs/cloudbuild.yaml

# Create Commit
git add notebooks/kubeflow_pipelines/cicd/labs/cloudbuild.yaml
git commit -m 'modified cloudbuild.yaml'

# Push the commit to your repository
git push origin master # github authentication will be required
```

#### Step 2: Create a **Cloud Build** trigger

Connect the fork you created in the previous step to your Google Cloud project and create a trigger following the steps in the [Creating GitHub app trigger](https://cloud.google.com/cloud-build/docs/create-github-app-triggers) article. Use the following values on the **Edit trigger** form:

|Field|Value|
|-----|-----|
|Name|[YOUR TRIGGER NAME]|
|Description|[YOUR TRIGGER DESCRIPTION]|
|Event| Tag|
|Source| [YOUR FORK]|
|Tag (regex)|.\*|
|Build Configuration|Cloud Build configuration file (yaml or json)|
|Cloud Build configuration file location| ./notebooks/kubeflow_pipelines/cicd/labs/cloudbuild.yaml|


Use the following values for the substitution variables:

|Variable|Value|
|--------|-----|
|_BASE_IMAGE_NAME|base_image|
|_COMPONENT_URL_SEARCH_PREFIX|https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/|
|_ENDPOINT|[Your KFP proxy host]|
|_PIPELINE_DSL|covertype_training_pipeline.py|
|_PIPELINE_FOLDER|notebooks/kubeflow_pipelines/cicd/labs|
|_PIPELINE_NAME|covertype_training_deployment|
|_PIPELINE_PACKAGE|covertype_training_pipeline.yaml|
|_PYTHON_VERSION|3.7|
|_RUNTIME_VERSION|1.15|
|_TRAINER_IMAGE_NAME|trainer_image|
|_USE_KFP_SA|False|

### Trigger the build

To start an automated build [create a new release of the repo in GitHub](https://help.github.com/en/github/administering-a-repository/creating-releases). Alternatively, you can start the build by applying a tag using `git`. 
```
git tag [TAG NAME]
git push origin --tags
```


<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>